# Filtrado, agrupación y merge de datos

## Crear datasets relacionados

In [36]:
import pandas as pd
import numpy as np

In [37]:
# Dataset de ventas
ventas = pd.DataFrame({
    'id_venta': range(1, 11),
    'id_cliente': np.random.choice([1, 2, 3, 4, 5], 10),
    'id_producto': np.random.choice([101, 102, 103, 104], 10),
    'cantidad': np.random.randint(1, 5, 10),
    'fecha': pd.date_range('2024-01-01', periods=10, freq='D')
})


In [38]:
# Dataset de clientes
clientes = pd.DataFrame({
    'id_cliente': [1, 2, 3, 4, 5],
    'nombre': ['Ana', 'Carlos', 'María', 'Juan', 'Luis'],
    'ciudad': ['Madrid', 'Barcelona', 'Madrid', 'Valencia', 'Sevilla']
})


In [39]:
# Dataset de productos
productos = pd.DataFrame({
    'id_producto': [101, 102, 103, 104],
    'nombre': ['Laptop', 'Mouse', 'Teclado', 'Monitor'],
    'precio': [1200, 25, 80, 300],
    'categoria': ['Electrónica', 'Accesorios', 'Accesorios', 'Electrónica']
})

In [40]:
print("Datasets creados:")
print(f"Ventas: {ventas.shape}")
print(f"Clientes: {clientes.shape}")
print(f"Productos: {productos.shape}")

Datasets creados:
Ventas: (10, 5)
Clientes: (5, 3)
Productos: (4, 4)


## Filtrado avanzado con query()

In [41]:
# Ventas del mes actual con query
ventas_recientes = ventas.query('fecha >= "2024-01-05"')
print(f"\nVentas recientes: {len(ventas_recientes)}")



Ventas recientes: 6


In [42]:
# Productos caros usando variable externa
precio_limite = 100
productos_caros = productos.query('precio >= @precio_limite')
print(f"Productos caros (>= {precio_limite}): {productos_caros['nombre'].tolist()}")

Productos caros (>= 100): ['Laptop', 'Monitor']


## Agrupación y agregación

In [43]:
# Ventas por producto
ventas_por_producto = ventas.groupby('id_producto')['cantidad'].sum()
print(f"\nVentas por producto:\n{ventas_por_producto}")



Ventas por producto:
id_producto
101    10
102     6
103     1
Name: cantidad, dtype: int32


In [44]:
# Estadísticas por cliente
stats_por_cliente = ventas.groupby('id_cliente').agg({
    'cantidad': ['sum', 'mean'],
    'id_venta': 'count'
})
print(f"\nEstadísticas por cliente:\n{stats_por_cliente}")


Estadísticas por cliente:
           cantidad           id_venta
                sum      mean    count
id_cliente                            
1                 8  2.000000        4
2                 3  1.500000        2
3                 1  1.000000        1
4                 5  1.666667        3


## Merge para análisis completo

In [45]:
# Unir ventas con productos
ventas_productos = pd.merge(ventas, productos, on='id_producto')

In [46]:
# Calcular totales
ventas_productos['total'] = ventas_productos['cantidad'] * ventas_productos['precio']

In [47]:
# Unir con clientes
analisis_completo = pd.merge(ventas_productos, clientes, on='id_cliente')
print(f"\nAnálisis completo (primeras 5 filas):\n{analisis_completo.head()}")


Análisis completo (primeras 5 filas):
   id_venta  id_cliente  id_producto  cantidad      fecha nombre_x  precio  \
0         1           1          101         4 2024-01-01   Laptop    1200   
1         2           1          101         1 2024-01-02   Laptop    1200   
2         3           3          101         1 2024-01-03   Laptop    1200   
3         4           2          102         2 2024-01-04    Mouse      25   
4         5           4          102         2 2024-01-05    Mouse      25   

     categoria  total nombre_y     ciudad  
0  Electrónica   4800      Ana     Madrid  
1  Electrónica   1200      Ana     Madrid  
2  Electrónica   1200    María     Madrid  
3   Accesorios     50   Carlos  Barcelona  
4   Accesorios     50     Juan   Valencia  


In [48]:
# Análisis por ciudad
ventas_por_ciudad = analisis_completo.groupby('ciudad')['total'].sum()
print(f"\nVentas totales por ciudad:\n{ventas_por_ciudad}")


Ventas totales por ciudad:
ciudad
Barcelona    1250
Madrid       7330
Valencia     3650
Name: total, dtype: int64


## Filtrado final

In [49]:
# Clientes con compras > 1000
# En `analisis_completo` el nombre del cliente quedó como 'nombre_y' tras el merge,
# por eso se usa ese nombre de columna para agrupar.
clientes_top = analisis_completo.groupby(['id_cliente', 'nombre_y'])['total'].sum()
clientes_top = clientes_top[clientes_top > 1000]
print(f"\nClientes con compras > 1000:\n{clientes_top}")


Clientes con compras > 1000:
id_cliente  nombre_y
1           Ana         6130
2           Carlos      1250
3           María       1200
4           Juan        3650
Name: total, dtype: int64
